In [ ]:
from dsds.prescreen import (
    remove_if_exists
    , regex_removal
    , var_removal
    , null_removal
    , unique_removal
    , constant_removal
    , date_removal
    , non_numeric_removal
    , get_unique_count
    , get_string_cols
)

from dsds.transform import (
    scale
    , impute
    , binary_encode
    , one_hot_encode
    , smooth_target_encode
    , ordinal_encode
    , ordinal_auto_encode
)

from dsds.fs import (
    mutual_info_selector
)

import polars as pl

In [ ]:
df = pl.scan_csv("../data/advertising.csv")

In [ ]:
df.limit(10).collect()

In [ ]:
blueprint = df.pipe(var_removal, threshold = 0.5, target = "Clicked on Ad")\
    .pipe(constant_removal)\
    .pipe(binary_encode)\
    .pipe(ordinal_auto_encode, cols = ["City", "Country"])\
    .pipe(impute, cols=["Daily Internet Usage", "Daily Internet Usage Band", "Area Income Band"], strategy="median")\
    .pipe(impute, cols=["Area Income"], strategy = "mean")\
    .pipe(scale, cols=["Area Income", "Daily Internet Usage"])\
    .pipe(one_hot_encode, cols= ["One_Hot_Test"])\
    .pipe(remove_if_exists, cols = ["Ad Topic Line", "Timestamp"])\
    .pipe(mutual_info_selector, target = "Clicked on Ad", top_k = 12)



In [ ]:
blueprint.collect()

In [ ]:
blueprint.write_json("pipe.json")

In [ ]:
f = open("pipe.json", "r")
json_str = f.read()
f.close()

new_lf = pl.scan_csv("../data/advertising.csv").from_json(json_str)

In [ ]:
new_lf.collect() == blueprint.collect()

In [ ]:
new_lf.collect()["Daily Internet Usage"].to_list()

In [ ]:
blueprint.collect()["Daily Internet Usage"].to_list()

In [ ]:
# Ok some small precision issues.. FFFFFFFFFFFFFFFFFFFF